In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , BatchNormalization

In [42]:
CIFAR10 = keras.datasets.cifar10

(x_train , y_train) , (x_test , y_test ) = CIFAR10.load_data()
x_train , x_test = x_train/255.0 , x_test/255.0

In [43]:
x_train[2].shape

(32, 32, 3)

# Adding batch size

In [44]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train , y_train)).batch(32).shuffle(buffer_size=1000)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test , y_test)).batch(32)

In [45]:
x_train.shape

(50000, 32, 32, 3)

# Dropout + Batch normalization
increasing accuracy and idea of layer's arrangement taken from :
https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

In [65]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv2D_1 = Conv2D(32 , (3,3) , activation="relu" , padding="same")
    self.conv2D_2 = Conv2D(32 , (3,3) , activation="relu" , padding="same")
    self.conv2D_3 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.conv2D_4 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.maxpool2D = MaxPooling2D()
    self.dropout = Dropout(0.2)
    self.dropout1 = Dropout(0.3)
    self.dropout2 = Dropout(0.4)
    self.dropout3 = Dropout(0.5)
    self.flatten = Flatten()
    self.dense1 = Dense(256 , activation="relu")
    self.dense = Dense(10 , activation="softmax")
    self.batchnorm1 = BatchNormalization()
    self.batchnorm2 = BatchNormalization()
    self.batchnorm3 = BatchNormalization()
    self.batchnorm4 = BatchNormalization()
    self.batchnorm7 = BatchNormalization()


  def call(self , x):
    x = self.conv2D_1(x)
    x = self.batchnorm1(x)
    x = self.conv2D_2(x)
    x = self.batchnorm2(x)
    x = self.maxpool2D(x)
    x = self.dropout(x)

    x = self.conv2D_3(x)
    x = self.batchnorm3(x)
    x = self.conv2D_4(x)
    x = self.batchnorm4(x)
    x = self.maxpool2D(x)
    x = self.dropout1(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.batchnorm7(x)
    x = self.dropout3(x)
    x = self.dense(x)
    return x

model = MyModel()

In [66]:
# Main loss function
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

In [67]:
# Metrics

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# Train

In [68]:
@tf.function
def train(images , labels):
    with tf.GradientTape() as tape :
      prediction = model(images)
      loss = loss_function(y_true=labels , y_pred=prediction)

    gradients = tape.gradient(loss , model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars= zip(gradients , model.trainable_variables))

    train_loss(loss)
    train_acc(labels, prediction)

In [69]:
@tf.function
def test(images , labels):
  prediction = model(images)
  loss = loss_function(y_true=labels , y_pred=prediction)
  test_loss(loss)
  test_acc(labels , prediction)

In [70]:
epochs = 5
for epoch in range(epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  #train
  for images , labels in train_dataset :
    train(images , labels)

  # test
  for images , labels in test_dataset :
    test(images , labels)


  print("epoch:" , epoch + 1  ,
        f"Train Loss: {train_loss.result()}" ,
        f"Train Acc: {train_acc.result()}" ,
        f"Test loss: {test_loss.result()}",
        f"Test Acc: {test_acc.result()}")

epoch: 1 Train Loss: 1.2230784893035889 Train Acc: 0.5640199780464172 Test loss: 1.0165948867797852 Test Acc: 0.642799973487854
epoch: 2 Train Loss: 0.7395774722099304 Train Acc: 0.7445399761199951 Test loss: 0.750623881816864 Test Acc: 0.7437999844551086
epoch: 3 Train Loss: 0.5330339670181274 Train Acc: 0.8191800117492676 Test loss: 0.7402235269546509 Test Acc: 0.7601000070571899
epoch: 4 Train Loss: 0.3924805521965027 Train Acc: 0.8687000274658203 Test loss: 0.9704635739326477 Test Acc: 0.7527999877929688
epoch: 5 Train Loss: 0.285096675157547 Train Acc: 0.9066600203514099 Test loss: 1.1440701484680176 Test Acc: 0.7712000012397766


# best test accuracy :  0.7712



# Note :    
after 5 epoch , test accuracy has been decreased .
maybe for this reason :    <br>

 increasing the number of epochs can lead to overfitting, where the model becomes too focused on the training data and performs poorly on new, unseen data.


In [71]:
model.save("cifar10_model")